In [21]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn import model_selection
import sklearn.metrics
import lightgbm as lgb
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import sklearn.metrics
import multiprocessing as mp
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [22]:
def get_train_id_with_label(time1, time2, time3, time4):
    # 获取训练集的用户id以及标签, 这里注册日志感觉需要两种处理，一种是1-n用来处理用户的，一种是真实的n1-n2,处理历史信息也就是特征的，两种放在不同的位置。。。。
    path = 'liwei/cuttime_data/'
    train_lau = pd.read_csv(path + str(time1) + '_' + str(time2) + '_app_launch_log.csv', usecols=['user_id'])
    train_reg = pd.read_csv(path + str(time1) + '_' + str(time2) + '_user_register_log.csv', usecols=['user_id'])
    train_vid = pd.read_csv(path + str(time1) + '_' + str(time2) + '_video_create_log.csv', usecols=['user_id'])
    train_act = pd.read_csv(path + str(time1) + '_' + str(time2) + '_user_activity_log.csv', usecols=['user_id'])

    train_user = np.unique(pd.concat([train_reg, train_lau, train_act, train_vid]))

    test_lau = pd.read_csv(path + str(time3) + '_' + str(time4) + '_app_launch_log.csv', usecols=['user_id'])
    test_reg = pd.read_csv(path + str(time3) + '_' + str(time4) + '_user_register_log.csv', usecols=['user_id'])
    test_vid = pd.read_csv(path + str(time3) + '_' + str(time4) + '_video_create_log.csv', usecols=['user_id'])
    test_act = pd.read_csv(path + str(time3) + '_' + str(time4) + '_user_activity_log.csv', usecols=['user_id'])

    test_user = np.unique(pd.concat([test_reg, test_lau, test_act, test_vid]))

    train_data = pd.DataFrame()
    train_data['user_id'] = train_user
    train_data['label'] = 0
    # commen_user = list(set(train_user).intersection(set(test_user)))
    train_data.loc[train_data['user_id'].isin(test_user), 'label'] = 1
    # for user in commen_user:
    # a = train_data[train_data.user_id == user].index
    # train_data.loc[a, 'label'] = 1
    return train_data

def get_test_id(time1, time2):
    # 获取测试集的用户id
    path = 'liwei/cuttime_data/'
    test_lau = pd.read_csv(path + str(time1) + '_' + str(time2) + '_app_launch_log.csv', usecols=['user_id'])
    test_reg = pd.read_csv(path + str(time1) + '_' + str(time2) + '_user_register_log.csv', usecols=['user_id'])
    test_vid = pd.read_csv(path + str(time1) + '_' + str(time2) + '_video_create_log.csv', usecols=['user_id'])
    test_act = pd.read_csv(path + str(time1) + '_' + str(time2) + '_user_activity_log.csv', usecols=['user_id'])
    test_user = np.unique(pd.concat([test_reg, test_lau, test_act, test_vid]))
    test_data = pd.DataFrame()
    test_data['user_id'] = test_user
    return test_data.user_id.tolist()

def get_lx_day(now):
    k1 = np.array(now)
    k2 = np.where(np.diff(k1) == 1)[0]
    i = 0
    ans = []
    while i < len(k2) - 1:
        l1 = 1
        while k2[i + 1] - k2[i] == 1:
            l1 += 1
            i += 1
            if i == len(k2) - 1:
                break
        if l1 == 1:
            i += 1
            ans.append(2)
        else:
            i += 1
            ans.append(l1 + 1)
    if len(k2) == 1:
        ans.append(2)
    if not ans:
        ans = [0]
    return ans

def get_interval_list(day_list):
    interval = pd.Series(sorted(day_list))
    interval = interval.diff(periods=1).tolist()
    interval = list(x for x in interval if str(x)!='nan')
    #interval = pd.Series(interval)
    return interval

In [23]:
def deal_act(path, time1, time2, user_id, save_path):
    act = pd.read_csv(path + str(time1) + '_' + str(time2) + '_user_activity_log.csv')
    gp = act[['user_id']].groupby(['user_id']).size().rename('total_act_times').reset_index()
    df=pd.DataFrame()
    df['user_id'] = user_id
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = act[['user_id','act_day']].groupby(['user_id','act_day']).size().rename('act_times_oneday').reset_index()
    gp1 = gp[['user_id','act_times_oneday']].groupby(['user_id'])['act_times_oneday'].max().rename('max_act_times').reset_index()
    gp2 = gp[['user_id','act_times_oneday']].groupby(['user_id'])['act_times_oneday'].min().rename('min_act_times').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    gp = act[['user_id','act_day']].groupby(['user_id',])['act_day'].nunique().rename('num_act_days').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = act[['user_id','act_day']].groupby(['user_id'])['act_day'].max().rename('last_act_days').reset_index()
    gp['last_act_days'] = time2 - gp['last_act_days']
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = act[['author_id']].groupby(['author_id']).size().rename('be_see_times').reset_index()
    gp.rename(columns={'author_id':'user_id'}, inplace=True)
    df = pd.merge(df, gp, on='user_id', how='left')
    df = df.fillna(0)
    
    gp = act[['user_id','author_id']].groupby(['user_id','author_id']).size().rename('num_authorid').reset_index()
    gp1 = gp[['user_id','num_authorid']].groupby(['user_id'])['num_authorid'].max().rename('max_num_authorid').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    
    gp = act[['user_id','video_id']].groupby(['user_id','video_id']).size().rename('num_videoid').reset_index()
    gp1 = gp[['user_id','num_videoid']].groupby(['user_id'])['num_videoid'].max().rename('max_num_videoid').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    
    gp = act[['user_id','act_page']]
    gp = pd.get_dummies(gp, columns = ['act_page'])
    gp = gp.groupby(['user_id']).sum().reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = act[['user_id','action_type']]
    gp = pd.get_dummies(gp, columns = ['action_type'])
    gp = gp.groupby(['user_id']).sum().reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = act[['user_id','act_day']].groupby(['user_id'])['act_day'].unique().rename('act_day_list').reset_index()
    gp['lx_list'] = gp['act_day_list'].apply(get_lx_day)
    gp['interval_list'] = gp['act_day_list'].apply(get_interval_list)
    gp['max_act_interval'] = gp['interval_list'].apply(lambda x : max(x) if x else 0)
    gp['min_act_interval'] = gp['interval_list'].apply(lambda x : min(x) if x else 0)
    gp['mean_act_interval'] = gp['interval_list'].apply(lambda x : np.mean(x) if x else 0)
    gp['var_act_interval'] = gp['interval_list'].apply(lambda x : np.var(x) if x else 0)
    gp['act_lx_len'] = gp['lx_list'].apply(lambda x : len(x))
    gp['act_lx_max'] = gp['lx_list'].apply(lambda x : max(x))
    gp['act_lx_min'] = gp['lx_list'].apply(lambda x : min(x))
    gp['act_lx_mean'] = gp['lx_list'].apply(lambda x : np.mean(x))
    gp['act_lx_var'] = gp['lx_list'].apply(lambda x : np.var(x))
    gp = gp.drop(['act_day_list','lx_list','interval_list'], axis = 1)
    df = pd.merge(df, gp, on='user_id', how='left')
    '''df['max_act_interval'] = 0
    df['min_act_interval'] = 0
    df['mean_act_interval'] = 0
    df['var_act_interval'] = 0
    df['fengdu_act_interval'] = 0
    df['piandu_act_interval'] = 0
    # 统计最后一次act的时间
    df['act_lx_len'] = 0
    df['act_lx_max'] = 0
    df['act_lx_min'] = 0
    df['act_lx_mean'] = 0
    df['act_lx_var'] = 0
    for user in tqdm(list(x for x in user_id if x in act['user_id'].unique())):
        single = act[act['user_id']==user]
        index = df[df.user_id==user].index
         #下面得到间隔列表
        interval = pd.Series(sorted(single['act_day'].unique()))
        interval = interval.diff(periods=1).tolist()
        interval = list(x for x in interval if str(x)!='nan')
        interval = pd.Series(interval)
        #下面获得间隔列表的均值，方差，最大值，最小值，峰度，偏度，最后一个、
        df.loc[index, 'max_act_interval'] = interval.max()
        df.loc[index, 'min_act_interval'] = interval.min()
        df.loc[index, 'mean_act_interval'] = interval.mean()
        df.loc[index, 'var_act_interval'] = inaterval.var()
        df.loc[index, 'fengdu_act_interval'] = interval.kurt()
        df.loc[index, 'piandu_act_interval'] = interval.skew()
        #处理连续天数这个特征
        act_day = sorted(single['act_day'].unique())
        lx = np.array(get_lx_day(act_day))
        df.loc[index,'act_lx_len'] = len(lx)
        df.loc[index,'act_lx_max'] = lx.max()
        df.loc[index,'act_lx_min'] = lx.min()
        df.loc[index,'act_lx_mean'] = lx.mean()
        df.loc[index,'act_lx_var'] = lx.var()'''
    df.fillna(0)
    df.to_csv(save_path, index=False)
    print(str(time1)+"-"+str(time2)+"activity feature finished!")

def deal_vid(path, time1, time2, user_id, save_path):
    vid = pd.read_csv(path + str(time1) + '_' + str(time2) + '_video_create_log.csv')
    gp = vid[['user_id']].groupby(['user_id']).size().rename('num_videos').reset_index()
    df=pd.DataFrame()
    df['user_id'] = user_id
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = vid.groupby(['user_id'])['video_day'].nunique().rename('num_videos_days').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = vid.groupby(['user_id'])['video_day'].value_counts().rename('counts').reset_index()
    gp1 = gp[['user_id','counts']].groupby(['user_id'])['counts'].max().rename('max_num_videos').reset_index()
    gp2 = gp[['user_id','counts']].groupby(['user_id'])['counts'].min().rename('min_num_videos').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    gp = vid.groupby(['user_id'])['video_day'].max().rename('last_video_time').reset_index()
    gp['last_video_time'] = 30 - gp['last_video_time']
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = vid[['user_id','video_day']].groupby(['user_id'])['video_day'].unique().rename('video_day_list').reset_index()
    gp['lx_list'] = gp['video_day_list'].apply(get_lx_day)
    gp['interval_list'] = gp['video_day_list'].apply(get_interval_list)
    gp['max_video_interval'] = gp['interval_list'].apply(lambda x : max(x) if x else 0)
    gp['min_video_interval'] = gp['interval_list'].apply(lambda x : min(x) if x else 0)
    gp['mean_video_interval'] = gp['interval_list'].apply(lambda x : np.mean(x) if x else 0)
    gp['var_video_interval'] = gp['interval_list'].apply(lambda x : np.var(x) if x else 0)
    gp['video_lx_len'] = gp['lx_list'].apply(lambda x : len(x))
    gp['video_lx_max'] = gp['lx_list'].apply(lambda x : max(x))
    gp['video_lx_min'] = gp['lx_list'].apply(lambda x : min(x))
    gp['video_lx_mean'] = gp['lx_list'].apply(lambda x : np.mean(x))
    gp['video_lx_var'] = gp['lx_list'].apply(lambda x : np.var(x))
    gp = gp.drop(['video_day_list','lx_list','interval_list'], axis = 1)
    df = pd.merge(df, gp, on='user_id', how='left')
    '''df['max_video_interval'] = 0
    df['min_video_interval'] = 0
    df['mean_video_interval'] = 0
    df['var_video_interval'] = 0
    df['fengdu_video_interval'] = 0
    df['piandu_video_interval'] = 0
    df['video_lx_len'] = 0
    df['video_lx_max'] = 0
    df['video_lx_min'] = 0
    df['video_lx_mean'] = 0
    df['video_lx_var'] = 0
    for user in tqdm(list(x for x in user_id if x in vid['user_id'].unique())):
        single = vid[vid['user_id']==user]
        index = df[df.user_id==user].index
        interval = pd.Series(sorted(single['video_day']))
        interval = interval.diff(periods=1).tolist()
        interval = list(x for x in interval if str(x)!='nan')
        interval = pd.Series(interval)
        #下面获得间隔列表的均值，方差，最大值，最小值，峰度，偏度，最后一个、
        df.loc[index, 'max_video_interval'] = interval.max()
        df.loc[index, 'min_video_interval'] = interval.min()
        df.loc[index, 'mean_video_interval'] = interval.mean()
        df.loc[index, 'var_video_interval'] = interval.var()
        df.loc[index, 'fengdu_video_interval'] = interval.kurt()
        df.loc[index, 'piandu_video_interval'] = interval.skew()
        #处理连续天数这个特征
        video_day = sorted(single['video_day'].unique())
        lx = np.array(get_lx_day(video_day))
        df.loc[index, 'video_lx_len'] = len(lx)
        df.loc[index, 'video_lx_max'] = lx.max()
        df.loc[index, 'video_lx_min'] = lx.min()
        df.loc[index, 'video_lx_mean'] = lx.mean()
        df.loc[index, 'video_lx_var'] = lx.var()'''
    df.fillna(0)
    df.to_csv(save_path, index=False)
    print(str(time1)+"-"+str(time2)+"video feature finished!")

def deal_reg(path, time1, time2, save_path):
    reg = pd.read_csv(path + str(time1) + '_' + str(time2) + '_user_register_log.csv')
    reg['device_type2']=reg['device_type']
    reg.loc[(reg['device_type']>10)&(reg['device_type']<=50) ,'device_type2']=11 
    reg.loc[(reg['device_type']>50)&(reg['device_type']<=100) ,'device_type2']=12
    reg.loc[(reg['device_type']>100)&(reg['device_type']<=200) ,'device_type2']=13
    reg.loc[(reg['device_type']>200)&(reg['device_type']<=400) ,'device_type2']=14
    reg.loc[(reg['device_type']>400) ,'device_type2']=15
    #把register_day分类处理，分为周末或非周末
    reg['register_day2'] = reg['register_day']
    reg.loc[reg['register_day'].isin([6,7,13,14,20,21,27,28]),'register_day2']=1
    reg.loc[~reg['register_day'].isin([6,7,13,14,20,21,27,28]),'register_day2']=0
    reg = pd.get_dummies(reg, columns=['register_type','device_type2'])
    reg = reg.drop(['register_day','device_type'],axis = 1)#这里设备类型可能是一个强特，需要处理，删除是不对滴。。。
    reg.to_csv(save_path, index=False)

    print(str(time1)+"-"+str(time2)+"register feature finished!")

def deal_launch(path, time1, time2, user_id, save_path):
    lau = pd.read_csv(path + str(time1) + '_' + str(time2) + '_app_launch_log.csv')
    gp = lau[['user_id']].groupby(['user_id']).size().rename('total_launch_times').reset_index()
    df=pd.DataFrame()
    df['user_id'] = user_id
    df = pd.merge(df, gp, on='user_id', how='left')
    gp = lau.groupby(['user_id'])['launch_day'].nunique().rename('num_launch_days').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = lau.groupby(['user_id'])['launch_day'].value_counts().rename('counts').reset_index()
    gp1 = gp[['user_id','counts']].groupby(['user_id'])['counts'].max().rename('max_launch_times').reset_index()
    gp2 = gp[['user_id','counts']].groupby(['user_id'])['counts'].min().rename('min_launch_times').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    gp = lau.groupby(['user_id'])['launch_day'].max().rename('last_launch_time').reset_index()
    gp['last_launch_time'] = 30 - gp['last_launch_time']
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = lau[['user_id','launch_day']].groupby(['user_id'])['launch_day'].unique().rename('launch_day_list').reset_index()
    gp['lx_list'] = gp['launch_day_list'].apply(get_lx_day)
    gp['interval_list'] = gp['launch_day_list'].apply(get_interval_list)
    gp['max_launch_interval'] = gp['interval_list'].apply(lambda x : max(x) if x else 0)
    gp['min_launch_interval'] = gp['interval_list'].apply(lambda x : min(x) if x else 0)
    gp['mean_launch_interval'] = gp['interval_list'].apply(lambda x : np.mean(x) if x else 0)
    gp['var_launch_interval'] = gp['interval_list'].apply(lambda x : np.var(x) if x else 0)
    gp['launch_lx_len'] = gp['lx_list'].apply(lambda x : len(x))
    gp['launch_lx_max'] = gp['lx_list'].apply(lambda x : max(x))
    gp['launch_lx_min'] = gp['lx_list'].apply(lambda x : min(x))
    gp['launch_lx_mean'] = gp['lx_list'].apply(lambda x : np.mean(x))
    gp['launch_lx_var'] = gp['lx_list'].apply(lambda x : np.var(x))
    gp = gp.drop(['launch_day_list','lx_list','interval_list'], axis = 1)
    df = pd.merge(df, gp, on='user_id', how='left')
    
    '''df['max_launch_interval'] = 0
    df['min_launch_interval'] = 0
    df['mean_launch_interval'] = 0
    df['var_launch_interval'] = 0
    df['fengdu_launch_interval'] = 0
    df['piandu_launch_interval'] = 0
    df['launch_lx_len'] = 0
    df['launch_lx_max'] = 0
    df['launch_lx_min'] = 0
    df['launch_lx_mean'] = 0
    df['launch_lx_var'] = 0
    for user in tqdm(list(x for x in user_id if x in lau['user_id'].unique())):
        single = lau[lau['user_id']==user]
        index = df[df.user_id==user].index
        #下面得到间隔列表
        interval = pd.Series(sorted(single['launch_day']))
        interval = interval.diff(periods=1).tolist()
        interval = list(x for x in interval if str(x)!='nan')
        interval = pd.Series(interval)
        #下面获得间隔列表的均值，方差，最大值，最小值，峰度，偏度，最后一个、
        df.loc[index, 'max_launch_interval'] = interval.max()
        df.loc[index, 'min_launch_interval'] = interval.min()
        df.loc[index, 'mean_launch_interval'] = interval.mean()
        df.loc[index, 'var_launch_interval'] = interval.var()
        df.loc[index, 'fengdu_launch_interval'] = interval.kurt()
        df.loc[index, 'piandu_launch_interval'] = interval.skew()
        #处理连续天数这个特征
        lau_day = sorted(single['launch_day'].unique())
        lx = np.array(get_lx_day(lau_day))
        df.loc[index,'launch_lx_len'] = len(lx)
        df.loc[index,'launch_lx_max'] = lx.max()
        df.loc[index,'launch_lx_min'] = lx.min()
        df.loc[index,'launch_lx_mean'] = lx.mean()
        df.loc[index,'launch_lx_var'] = lx.var()'''
    df.fillna(0)
    df.to_csv(save_path, index=False)
    print(str(time1)+"-"+str(time2)+"launch feature finished!")

def deal_other(path, time1, time2, user_id, save_path):
    lau = pd.read_csv(path + str(time1) + '_' + str(time2) + '_app_launch_log.csv')
    vid = pd.read_csv(path + str(time1) + '_' + str(time2) + '_video_create_log.csv')
    act = pd.read_csv(path + str(time1) + '_' + str(time2) + '_user_activity_log.csv')
    days2_act = pd.concat([act[act['act_day']==time2],act[act['act_day']==(time2-1)]])
    days2_vid = pd.concat([vid[vid['video_day']==time2],vid[vid['video_day']==(time2-1)]])
    days2_lau = pd.concat([lau[lau['launch_day']==time2],lau[lau['launch_day']==(time2-1)]])
    
    gp = days2_act[['user_id']].groupby(['user_id']).size().rename('2day_act_times').reset_index()
    df=pd.DataFrame()
    df['user_id'] = user_id
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = days2_act[['user_id','act_page']]
    gp.rename(columns={'act_page':'days2_act_page'}, inplace = True)
    gp = pd.get_dummies(gp, columns = ['days2_act_page'])
    gp = gp.groupby(['user_id']).sum().reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = days2_act[['user_id','action_type']]
    gp.rename(columns={'action_type':'days2_action_type'}, inplace = True)
    gp = pd.get_dummies(gp, columns = ['days2_action_type'])
    gp = gp.groupby(['user_id']).sum().reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = days2_vid[['user_id']].groupby(['user_id']).size().rename('2day_video_times').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = days2_lau[['user_id']].groupby(['user_id']).size().rename('2day_launch_times').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    fir_week_lau = lau[(lau['launch_day']>=time1)&(lau['launch_day']<=(time1+7))]
    sec_week_lau = lau[(lau['launch_day']>=(time1+8))&(lau['launch_day']<=time2)]
    
    gp = fir_week_lau[['user_id']].groupby(['user_id']).size().rename('fir_week_lau_total_times').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    gp = fir_week_lau.groupby(['user_id'])['launch_day'].value_counts().rename('counts').reset_index()
    gp1 = gp[['user_id','counts']].groupby(['user_id'])['counts'].max().rename('fir_week_lau_max_times').reset_index()
    gp2 = gp[['user_id','counts']].groupby(['user_id'])['counts'].min().rename('fir_week_lau_min_times').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    gp = sec_week_lau[['user_id']].groupby(['user_id']).size().rename('sec_week_lau_total_times').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    gp = sec_week_lau.groupby(['user_id'])['launch_day'].value_counts().rename('counts').reset_index()
    gp1 = gp[['user_id','counts']].groupby(['user_id'])['counts'].max().rename('sec_week_lau_max_times').reset_index()
    gp2 = gp[['user_id','counts']].groupby(['user_id'])['counts'].min().rename('sec_week_lau_min_times').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    fir_week_act = act[(act['act_day']>=time1)&(act['act_day']<=(time1+7))]
    sec_week_act = act[(act['act_day']>=(time1+8))&(act['act_day']<=time2)]
    
    gp = fir_week_act[['user_id']].groupby(['user_id']).size().rename('fir_week_act_total_times').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = fir_week_act[['user_id','act_day']].groupby(['user_id','act_day']).size().rename('act_times_oneday').reset_index()
    gp1 = gp[['user_id','act_times_oneday']].groupby(['user_id'])['act_times_oneday'].max().rename('fir_week_act_max_times').reset_index()
    gp2 = gp[['user_id','act_times_oneday']].groupby(['user_id'])['act_times_oneday'].min().rename('fir_week_act_min_times').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    gp = sec_week_act[['user_id']].groupby(['user_id']).size().rename('sec_week_act_total_times').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = sec_week_act[['user_id','act_day']].groupby(['user_id','act_day']).size().rename('act_times_oneday').reset_index()
    gp1 = gp[['user_id','act_times_oneday']].groupby(['user_id'])['act_times_oneday'].max().rename('sec_week_act_max_times').reset_index()
    gp2 = gp[['user_id','act_times_oneday']].groupby(['user_id'])['act_times_oneday'].min().rename('sec_week_act_min_times').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    fir_week_vid = vid[(vid['video_day']>=time1)&(vid['video_day']<=(time1+7))]
    sec_week_vid = vid[(vid['video_day']>=(time1+8))&(vid['video_day']<=time2)]
    
    gp = fir_week_vid[['user_id']].groupby(['user_id']).size().rename('fir_week_vid_total_times').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = fir_week_vid.groupby(['user_id'])['video_day'].value_counts().rename('counts').reset_index()
    gp1 = gp[['user_id','counts']].groupby(['user_id'])['counts'].max().rename('fir_week_vid_max_times').reset_index()
    gp2 = gp[['user_id','counts']].groupby(['user_id'])['counts'].min().rename('fir_week_vid_min_times').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    gp = sec_week_vid[['user_id']].groupby(['user_id']).size().rename('sec_week_vid_total_times').reset_index()
    df = pd.merge(df, gp, on='user_id', how='left')
    
    gp = sec_week_vid.groupby(['user_id'])['video_day'].value_counts().rename('counts').reset_index()
    gp1 = gp[['user_id','counts']].groupby(['user_id'])['counts'].max().rename('sec_week_vid_max_times').reset_index()
    gp2 = gp[['user_id','counts']].groupby(['user_id'])['counts'].min().rename('sec_week_vid_min_times').reset_index()
    df = pd.merge(df, gp1, on='user_id', how='left')
    df = pd.merge(df, gp2, on='user_id', how='left')
    
    df.fillna(0)
    df.to_csv(save_path, index=False)
    print(str(time1)+"-"+str(time2)+"other feature is ok")

In [25]:
arg1 = "liwei/cuttime_data/"
arg2 = 8
arg3 = 23
arg4 = get_test_id(8, 23)
arg_p1 = "liwei/cuttime_data/"+'train_8_23_vid.csv'
arg_p2 = "liwei/cuttime_data/"+'train_8_23_reg.csv'
arg_p3 = "liwei/cuttime_data/"+'train_8_23_laun.csv'
arg_p4 = "liwei/cuttime_data/"+'train_8_23_act.csv'
arg_p5 = "liwei/cuttime_data/"+'train_8_23_out.csv'

arg2_2 = 1
arg3_2 = 16
arg4_2 = get_test_id(1, 16)
arg_p1_2 = "liwei/cuttime_data/"+'train_1_16_vid.csv'
arg_p2_2 = "liwei/cuttime_data/"+'train_1_16_reg.csv'
arg_p3_2 = "liwei/cuttime_data/"+'train_1_16_laun.csv'
arg_p4_2 = "liwei/cuttime_data/"+'train_1_16_act.csv'
arg_p5_2 = "liwei/cuttime_data/"+'train_1_16_out.csv'

arg2_3 = 15
arg3_3 = 30
arg4_3 = get_test_id(15, 30)
arg_p1_3 = "liwei/cuttime_data/"+'train_15_30_vid.csv'
arg_p2_3 = "liwei/cuttime_data/"+'train_15_30_reg.csv'
arg_p3_3 = "liwei/cuttime_data/"+'train_15_30_laun.csv'
arg_p4_3 = "liwei/cuttime_data/"+'train_15_30_act.csv'
arg_p5_3 = "liwei/cuttime_data/"+'train_15_30_out.csv'

In [66]:
p1 = mp.Process(target=deal_vid,args=(arg1,arg2,arg3,arg4,arg_p1))
p2 = mp.Process(target=deal_vid,args=(arg1,arg2_2,arg3_2,arg4_2,arg_p1_2))
p3 = mp.Process(target=deal_vid,args=(arg1,arg2_3,arg3_3,arg4_3,arg_p1_3))
p4 = mp.Process(target=deal_reg,args=(arg1,arg2,arg3,arg_p2))
p5 = mp.Process(target=deal_reg,args=(arg1,arg2_2,arg3_2,arg_p2_2))
p6 = mp.Process(target=deal_reg,args=(arg1,arg2_3,arg3_3,arg_p2_3))
p1.start()
p2.start()
p3.start()
p4.start()
p5.start()
p6.start()
p1.join()
p2.join()
p3.join()
p4.join()
p5.join()
p6.join()

1-16register feature finished!
8-23register feature finished!
1-16video feature finished!
15-30register feature finished!
8-23video feature finished!
15-30video feature finished!


In [67]:
p1 = mp.Process(target=deal_launch,args=(arg1,arg2,arg3,arg4,arg_p3))
p2 = mp.Process(target=deal_launch,args=(arg1,arg2_2,arg3_2,arg4_3,arg_p3_2))
p3 = mp.Process(target=deal_launch,args=(arg1,arg2_3,arg3_3,arg4_3,arg_p3_3))
p4 = mp.Process(target=deal_act,args=(arg1,arg2,arg3,arg4,arg_p4))
p5 = mp.Process(target=deal_act,args=(arg1,arg2_2,arg3_2,arg4_3,arg_p4_2))
p6 = mp.Process(target=deal_act,args=(arg1,arg2_3,arg3_3,arg4_3,arg_p4_3))
p1.start()
p2.start()
p3.start()
p4.start()
p5.start()
p6.start()
p1.join()
p2.join()
p3.join()
p4.join()
p5.join()
p6.join()

1-16launch feature finished!
8-23launch feature finished!
15-30launch feature finished!
1-16activity feature finished!
15-30activity feature finished!


In [ ]:
uid = get_test_id(8, 23)
deal_act("liwei/cuttime_data/", 8, 23, uid, "liwei/train_test_data/train_8_23_act.csv")

8-23activity feature finished!


In [26]:
uid = get_test_id(8, 23)
deal_other("liwei/cuttime_data/", 8, 23, uid, "liwei/cuttime_data/"+'train_8_23_out.csv')
uid = get_test_id(1, 16)
deal_other("liwei/cuttime_data/", 1, 16, uid, "liwei/cuttime_data/"+'train_1_16_out.csv')
uid = get_test_id(15, 30)
deal_other("liwei/cuttime_data/", 15, 30, uid, "liwei/cuttime_data/"+'train_15_30_out.csv')

/opt/conda/lib/python3.5/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


8-23other feature is ok
1-16other feature is ok
15-30other feature is ok


In [22]:
uid = get_test_id(15, 30)
deal_other("liwei/cuttime_data/", 15, 30, uid, "liwei/cuttime_data/"+'train_15_30_out.csv')

15-30other feature is ok


In [27]:
def concat_feature(fileList,save_path):
    for i,fileName in enumerate(fileList):
        if i==0:
            train_feature = pd.read_csv(fileName)
        else:
            tmp = pd.read_csv(fileName)

            train_feature = pd.merge(train_feature, tmp,on='user_id',how="left")
    train_feature.fillna(0, inplace=True)
    train_feature.to_csv(save_path, index=False)
    print("merge feature is ok")
    
flist1 = [arg_p1,arg_p2,arg_p3,arg_p4,arg_p5]
flist2 = [arg_p1_2,arg_p2_2,arg_p3_2,arg_p4_2,arg_p5_2]
flist3 = [arg_p1_3,arg_p2_3,arg_p3_3,arg_p4_3,arg_p5_3]
save_path1 = "liwei/train_test_data/train_8_23.csv"
save_path2 = "liwei/train_test_data/train_1_16.csv"
save_path3 = "liwei/train_test_data/test.csv"

p1 = mp.Process(target=concat_feature,args=(flist1,save_path1))
p2 = mp.Process(target=concat_feature,args=(flist2,save_path2))
p3 = mp.Process(target=concat_feature,args=(flist3,save_path3))
p1.start()
p2.start()
p3.start()
p1.join()
p2.join()
p3.join()

merge feature is ok
merge feature is ok
merge feature is ok


In [28]:
path = 'liwei/train_test_data/'
train_data_1_16 = get_train_id_with_label(1,16,17,23)
train_data_8_23 = get_train_id_with_label(8,23,24,30)

train_1_16 = pd.read_csv("liwei/train_test_data/train_1_16.csv")
train_1_16 = pd.merge(train_1_16,train_data_1_16,on="user_id")
print(train_1_16.shape)
train_8_23 = pd.read_csv("liwei/train_test_data/train_8_23.csv")
train_8_23 = pd.merge(train_8_23,train_data_8_23,on="user_id")
print(train_8_23.shape)
train_feature = pd.concat([train_1_16, train_8_23])
print(train_feature.shape)
train_feature.to_csv(path+'train.csv', index=False)
print("train data saved!")

del train_feature

(264492, 119)
(413752, 119)
(678244, 119)
train data saved!


In [33]:
def main():
    def xgb_model(train,test):
        print("Xgb begin to training...")
        label = train.pop('label')
        xgb1 = XGBClassifier(learning_rate =0.01,
                             n_estimators=2000,
                             max_depth=5,
                             min_child_weight=1,
                             gamma=0,
                             subsample=0.8,
                             colsample_bytree=0.8,
                             reg_alpha=0,
                             objective= 'binary:logistic',
                             nthread=8,
                             scale_pos_weight=1,
                             seed=27)
        modelfit(xgb1, train, label,test)# this module conclude that the best estimator is 57
    def modelfit(alg, dtrain, label,test, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
        if useTrainCV:
            xgb_param = alg.get_xgb_params()
            xgtrain = xgb.DMatrix(dtrain.values, label=label)
            cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                metrics='auc', early_stopping_rounds=early_stopping_rounds)
            print("The best estimatoes is %s"%(cvresult.shape[0]))
            alg.set_params(n_estimators=cvresult.shape[0])
    
        #Fit the algorithm on the data
        alg.fit(dtrain, label ,eval_metric='auc')
    
        #Predict training set:
        dtrain_predictions = alg.predict(dtrain)
        dtrain_predprob = alg.predict_proba(dtrain)[:,1]
    
        #Print model report:
        print ("\nModel Report")
        print ("Accuracy : %.4g" % metrics.accuracy_score(label, dtrain_predictions))
        #print ("F1_score : %.4g" % metrics.f1_score(label, dtrain_predictions))
        print ("AUC Score (Train): %f" % metrics.roc_auc_score(label, dtrain_predprob))
        ###################################################
        compare = pd.DataFrame()                     ######
        compare['label'] = label                     ######
        compare['predicton'] = dtrain_predictions    ######
        compare.to_csv('compare_xgb.csv',index=False)######
        ###################################################
        result = pd.DataFrame()
        result['user_id'] = test['user_id']
        test = test.drop(['user_id'],axis=1)
        result['proba'] = alg.predict_proba(test)[:,1]
        result.to_csv('xgboutput-feat_all.csv', index=False, header=None)
        
    def lgb_model(train_data, test):
        print('lgb begin to training...')
        label = train_data.pop('label')
        X_train, X_test, Y_train, Y_test = model_selection.train_test_split(train_data, label, test_size=0.2, random_state=2018)
        lgb_train = lgb.Dataset(X_train, Y_train)
        lgb_eval =  lgb.Dataset(X_test,  Y_test)
        
        '''params = {
            'learning_rate':0.03,
            'max_depth':6,
    	'num_leaves':52,
            'min_data_in_leaf':25,
            'task':'train',
            'boosting_type':'gbdt',
            'objective':'binary',
            'metric':{'auc', 'binary_logloss'}
        }'''
        params = {
            'learning_rate':0.01,
            'max_depth':5,
            'task':'train',
            'boosting_type':'gbdt',
            'objective':'binary',
            'metric':{'auc', 'binary_logloss'}
        }
        gbm = lgb.train(params, lgb_train, num_boost_round=1500, valid_sets=lgb_eval)
        
        temp = gbm.predict(X_test)
        print ("\nModel Report")
        #print ("Accuracy : %.4g" % metrics.accuracy_score(Y_test, temp))
        print ("AUC Score (eval): %f" % metrics.roc_auc_score(Y_test, temp))
        result = pd.DataFrame()
        result['user_id'] = test['user_id']
        test = test.drop(['user_id'],axis=1)
        result['proba'] = gbm.predict(test)
        result.to_csv('lgboutput.csv', index=False, header=None)
        print('important feature: ' + str(list(gbm.feature_importance())))
    ###############################################################################################
    train_data = pd.read_csv('liwei/train_test_data/train.csv')
    test_data  = pd.read_csv('liwei/train_test_data/test.csv')

    train_data = train_data.fillna(0)
    test_data = test_data.fillna(0)
    
    normal_feature = ['num_videos','num_videos_days','max_num_videos','min_num_videos','last_video_time',
'max_video_interval','min_video_interval','mean_video_interval','var_video_interval','video_lx_len','video_lx_max',
'video_lx_min','video_lx_mean','video_lx_var','register_day2','register_type_0','register_type_1','register_type_2',
'register_type_3','register_type_4','register_type_5','register_type_6','register_type_7','register_type_8',
'register_type_9','register_type_10','register_type_11','device_type2_0','device_type2_1','device_type2_2',
'device_type2_3','device_type2_4','device_type2_5','device_type2_6','device_type2_7','device_type2_8','device_type2_9',
'device_type2_10','device_type2_11','device_type2_12','device_type2_13','device_type2_14','device_type2_15','total_launch_times',
'num_launch_days','max_launch_times','min_launch_times','last_launch_time','max_launch_interval','min_launch_interval',
'mean_launch_interval','var_launch_interval','launch_lx_len','launch_lx_max','launch_lx_min','launch_lx_mean','launch_lx_var',
'total_act_times','max_act_times','min_act_times','num_act_days','last_act_days','be_see_times','max_num_authorid','max_num_videoid',
'act_page_0','act_page_1','act_page_2','act_page_3','act_page_4','action_type_0','action_type_1','action_type_2','action_type_3',
'action_type_4','action_type_5','max_act_interval','min_act_interval','mean_act_interval','var_act_interval','act_lx_len',
'act_lx_max','act_lx_min','act_lx_mean','act_lx_var','2day_act_times','days2_act_page_0','days2_act_page_1','days2_act_page_2',
'days2_act_page_3','days2_act_page_4','days2_action_type_0','days2_action_type_1','days2_action_type_2','days2_action_type_3',
'days2_action_type_4','days2_action_type_5','2day_video_times','2day_launch_times','fir_week_lau_total_times','fir_week_lau_max_times',
'fir_week_lau_min_times','sec_week_lau_total_times','sec_week_lau_max_times','sec_week_lau_min_times','fir_week_act_total_times',
'fir_week_act_max_times','fir_week_act_min_times','sec_week_act_total_times','sec_week_act_max_times','sec_week_act_min_times',
'fir_week_vid_total_times','fir_week_vid_max_times','fir_week_vid_min_times','sec_week_vid_total_times','sec_week_vid_max_times',
'sec_week_vid_min_times']
    for feature in normal_feature:
        mini = train_data[feature].min()
        maxi = train_data[feature].max()
        train_data[feature] = (train_data[feature]-mini)/(maxi-mini)
    for feature in normal_feature:
        mini = test_data[feature].min()
        maxi = test_data[feature].max()
        test_data[feature] = (test_data[feature]-mini)/(maxi-mini)
    train_data = train_data.drop(['user_id'],axis=1)
    xgb_model(train_data, test_data)
    #lgb_model(train_data, test_data)

In [41]:
#xgb,使用全部的特征，107维
main()

Xgb begin to training...


In [35]:
main()
#lgb,使用全部特征

lgb begin to training...
[1]	valid_0's binary_logloss: 0.688746	valid_0's auc: 0.879143
[2]	valid_0's binary_logloss: 0.684421	valid_0's auc: 0.879677
[3]	valid_0's binary_logloss: 0.68018	valid_0's auc: 0.879908
[4]	valid_0's binary_logloss: 0.676034	valid_0's auc: 0.879865
[5]	valid_0's binary_logloss: 0.671956	valid_0's auc: 0.880017
[6]	valid_0's binary_logloss: 0.667965	valid_0's auc: 0.880433
[7]	valid_0's binary_logloss: 0.664028	valid_0's auc: 0.881266
[8]	valid_0's binary_logloss: 0.66018	valid_0's auc: 0.881167
[9]	valid_0's binary_logloss: 0.656391	valid_0's auc: 0.881373
[10]	valid_0's binary_logloss: 0.652672	valid_0's auc: 0.881392
[11]	valid_0's binary_logloss: 0.649052	valid_0's auc: 0.881439
[12]	valid_0's binary_logloss: 0.64547	valid_0's auc: 0.881429
[13]	valid_0's binary_logloss: 0.641956	valid_0's auc: 0.881532
[14]	valid_0's binary_logloss: 0.638517	valid_0's auc: 0.881559
[15]	valid_0's binary_logloss: 0.63513	valid_0's auc: 0.881542
[16]	valid_0's binary_loglos

In [38]:
result_xgb = pd.read_csv('xgboutput-feat_all.csv', header=None, names=["user_id","result_xgb"])
result_lgb = pd.read_csv('lgboutput.csv',header=None, names=["user_id","result_lgb"])
result = pd.merge(result_xgb, result_lgb, on='user_id', how='left')
result['pred'] = result['result_xgb']*0.5 + result['result_lgb']*0.5
result = result[['user_id', 'pred']]
result.to_csv('result_merge.csv', header=None, index=False)

In [39]:
!./kesci_submit -token add5552ceb5ba48f -file result_merge.csv

Kesci Submit Tool
Working...
Success.
OK